# GCCompany Demo: Vector Searches with Amazon OpenSearch


## 필요한 패키지 설치


In [1]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3

## 데이터 준비


In [3]:
import pandas as pd
import requests

hotels_raw_df = pd.read_csv("../data/kr_hotels_korean_final.csv", low_memory=False)
hotels_raw_df.head(10)

,name,city,rating,address,description,facilities,attractions,airport,text,map
0,갤러리아 호텔,안동시,2,경상북도 안동시 동헌길 23,"안동 운흥동 5층 석탑 인근에 위치한 호텔입니다. 안동 구시가와 가깝고, 달밤다리와...","에어컨, 무선인터넷, 유선인터넷, 케이블TV, 냉장고, 커피/차 메이커, 헤어드라이...","운흥동 5층 석탑(0.4km), 안동 구시가(0.6km), 안동문화예술의전당(1.1...",정보가 업데이트 되지 않았습니다,갤러리아 호텔은 안동시에 위치한 2성급 호텔입니다. 안동 운흥동 5층 석탑과 구시가...,"36.564355,128.734417"
1,윈 호텔,안동,2,경상북도 안동시 옥동면 옥동길 28,안동 윈 호텔은 안동시에서 비즈니스와 레저를 위해 완벽한 위치에 있습니다. 이 호텔...,스파,정보가 업데이트 되지 않았습니다,정보가 업데이트 되지 않았습니다,안동 윈 호텔은 안동시에 위치한 2성급 호텔입니다. 경상북도 안동시 옥동면 옥동길 ...,"36.564552,128.702543"
2,안동 리첼 호텔,안동,3,경상북도 안동시 광황당지로 346-69,"안동 민속박물관 근처에 위치한 호텔입니다. 안동 하회 마을에서 38.9km, 신서동...","레스토랑, 조식 뷔페, 무료 와이파이, 케이블 TV","신서동 7층 석탑(12km), 안동 운흥동 5층 석탑(13km), 안동 구시장(13...",대구국제공항(106.5km),안동 리첼 호텔은 경상북도 안동시에 위치한 3성급 호텔입니다. 안동 민속박물관 근처...,"36.567943,128.781554"
3,페터팬 게스트하우스,안동시,1,경상북도 안동시 안흥동 36698 안흥동 312-2 2층,안동 전통시장에서 313미터 떨어진 곳에 위치한 이 게스트하우스는 유네스코 세계문화...,주방,주요 관광지,대구국제공항,"페터팬 게스트하우스는 안동 문화관광단지와 안동버스터미널에서 13분 거리에 있으며, ...","36.56374,128.72427"
4,안동호텔,안동,3,경상북도 안동시 문화광장길 40-11,안동 안흥동 오층석탑 근처에 위치해 있습니다. 안동 구시장과 안동 안흥동 오층석탑에...,"에어컨, 냉장고, LCD TV, 무선 인터넷, 케이블 TV, 욕실, 책상, 커피/차...","안동 구시장(0.3km), 안동 안흥동 오층석탑(0.4km), 안동 천주교회(0.6...",대구국제공항,안동호텔은 안동시에 위치한 3성급 호텔입니다. 안동 구시장과 안동 안흥동 오층석탑 ...,"36.564737,128.731612"
5,고려호텔,안동시,2,경상북도 안동시 760-050 동부동 1가 116번지,안동 운흥동 오층석탑 인근에 위치하고 있습니다. 안동 구시장과 가까운 거리에 있으며...,"에어컨, 무료 유선 및 무선 인터넷","안동 운흥동 오층석탑(0.2km), 안동 구시장(0.3km), 안동 가톨릭교회(0....",정보가 업데이트 되지 않았습니다,안동시에 위치한 고려호텔은 2성급 호텔입니다. 안동 운흥동 오층석탑과 안동 구시장 ...,"36.564397,128.733147"
6,안동 파크 호텔,안동시,3,경상북도 안동시 36705 운흥동 324,헤드라인: 임청각 가까이에 위치함 위치: 안동 파크 호텔에 머무시면 안동 중심가에 ...,"냉장고, LCD TV, 무료 유선 및 무선 인터넷, 대형 욕조, 헤어드라이어, 무료...","안동 운흥동 오층석탑(0.4km), AMF 볼링센터(0.5km), 안동 구시장(0....",대구국제공항,"안동 파크 호텔은 안동시 중심가에 위치해 있으며 임청각, 안동 운흥동 오층석탑, A...","36.563582,128.735469"
7,안동풍경게스트하우스 호스텔,안동,2,안동시 노하2길 19-5,헤드라인: 교외에 위치해 있습니다. 위치: 안동에 위치한 안동 풍경 호스텔 앤 라이...,"무료 무선 인터넷, 에어컨, 개인 욕실, 암막 커튼, 매일 청소 서비스","안동 가톨릭 교회(5.1km), 안동 구 시장(5.1km), 안동 문화예술회관(5....",대구국제공항(98.5km),"안동 풍경 게스트하우스 호스텔은 안동시 교외에 위치해 있으며, 무료 무선 인터넷, ...","36.578108,128.680536"
8,링커 파티 하우스 호스텔,안동,2,안동 대안로 180-1,안동 운흥동 오층석탑 인근에 위치해 있습니다. 링커 파티 하우스 - 호스텔에 머무르...,"공용 주방, 무료 와이파이","안동 운흥동 오층석탑(0.2km), 안동 구시장(0.3km), 안동문화예술의전당(0...",정보가 업데이트 되지 않았습니다,"안동에 위치한 2성급 호스텔 링커 파티 하우스 호스텔은 안동 운흥동 오층석탑, 안동...","36.563936,128.732532"
9,고타야 게스트하우스,안동,2,안동 동흥 1길 20,고타야 게스트하우스는 안동역에서 250미터 거리에 위치해 있습니다. 이 휴가용 주택...,"공유 주방, 공유 라운지, TV 공간, 무료 WiFi, 평면 케이블 TV, 세탁기,...",주요 관광지,정보가 업데이트 되지 않았습니다,"고타야 게스트하우스는 안동역에서 250미터 거리에 위치해 있으며, 공유 주방, 공유...","36.56461,128.73433"


In [4]:
hotels_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11514 entries, 0 to 11513
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         11514 non-null  object
 1   city         11514 non-null  object
 2   rating       11514 non-null  int64 
 3   address      11514 non-null  object
 4   description  11514 non-null  object
 5   facilities   10981 non-null  object
 6   attractions  11495 non-null  object
 7   airport      11218 non-null  object
 8   summary      11514 non-null  object
 9   map          11514 non-null  object
dtypes: int64(1), object(9)
memory usage: 899.7+ KB


## OpenSearch 클라이언트 생성


In [1]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import json

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region="us-east-1")

aos_host = "search-bedrock-opensearch-6gczufsfv5pp76bvtsd62qkrxu.aos.us-east-1.on.aws"


aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=600,
)

## Neural(Semantic) Search Embedding Model 준비


### ML 플러그인 설정


In [ ]:
s = b'{"transient":{"plugins.ml_commons.only_run_on_ml_node": false}}'
aos_client.cluster.put_settings(body=s)

{'acknowledged': True,
 'persistent': {},
 'transient': {'plugins': {'ml_commons': {'only_run_on_ml_node': 'false'}}}}

### Bedrock Embedding Model Connector 생성


In [5]:
base_url = (
    "https://search-bedrock-opensearch-6gczufsfv5pp76bvtsd62qkrxu.aos.us-east-1.on.aws/_plugins/_ml"
)

In [ ]:
# Bedrock Titan Embeddings G1 - Text Connector 생성

connector_payload = {
    "name": "Connector to Bedrock Titan Embeddings G1 - Text",
    "description": "The connector to Bedrock Titan Embeddings G1 - Text",
    "version": 1,
    "protocol": "aws_sigv4",
    "parameters": {
        "region": "us-east-1",
        "service_name": "bedrock",
        "input_docs_processed_step_size": 2,
    },
    "credential": {"roleArn": "arn:aws:iam::236241703319:role/OpenSearchForBedrockRole"},
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "url": "https://bedrock-runtime.us-east-1.amazonaws.com/model/amazon.titan-embed-text-v1/invoke",
            "headers": {"content-type": "application/json", "x-amz-content-sha256": "required"},
            "request_body": '{ "inputText": "${parameters.inputText}" }',
            "pre_process_function": '\n    StringBuilder builder = new StringBuilder();\n    builder.append("\\"");\n    String first = params.text_docs[0];\n    builder.append(first);\n    builder.append("\\"");\n    def parameters = "{" +"\\"inputText\\":" + builder + "}";\n    return  "{" +"\\"parameters\\":" + parameters + "}";',
            "post_process_function": '\n      def name = "sentence_embedding";\n      def dataType = "FLOAT32";\n      if (params.embedding == null || params.embedding.length == 0) {\n        return params.message;\n      }\n      def shape = [params.embedding.length];\n      def json = "{" +\n                 "\\"name\\":\\"" + name + "\\"," +\n                 "\\"data_type\\":\\"" + dataType + "\\"," +\n                 "\\"shape\\":" + shape + "," +\n                 "\\"data\\":" + params.embedding +\n                 "}";\n      return json;\n    ',
        }
    ],
}

# Send the request
response = requests.post(
    base_url + "/connectors/_create",
    auth=auth,
    json=connector_payload,
)

# Print the response
print(response.text)

{"connector_id":"iD1Qjo4B1FbzESchRpWr"}


In [ ]:
obj = json.loads(response.text)
connector_id = obj["connector_id"]

### 모델 그룹 등록


In [ ]:
register_model_group_payload = {
    "name": "Bedrock Models",
    "description": "This is a model group for Amazon Bedrock remote model.",
}

# Send the request
response = requests.post(
    base_url + "/model_groups/_register",
    auth=auth,
    json=register_model_group_payload,
)

# Print the response
print(response.text)

{"model_group_id":"EVZQjo4B5vU2gwgcSrgY","status":"CREATED"}


### 모델 생성


In [ ]:
register_model_payload = {
    "name": "Bedrock text embedding model",
    "function_name": "remote",
    "model_group_id": model_group_id,
    "description": "This is Bedrock Titan Embeddings G1 - Text model",
    "connector_id": "flaMhY4B5vU2gwgcXTu4",
}

# Send the request
response = requests.post(
    base_url + "/models/_register",
    auth=auth,
    json=register_model_payload,
)

# Print the response
print(response.text)

{"task_id":"iT1Qjo4B1FbzESchTZWD","status":"CREATED","model_id":"ij1Qjo4B1FbzESchTZWr"}


In [ ]:
obj = json.loads(response.text)
model_id = obj["model_id"]
print(model_id)

ij1Qjo4B1FbzESchTZWr


### 모델 배포


In [ ]:
deploy_model_payload = {}

# Send the request
response = requests.post(
    base_url + "/models/" + model_id + "/_deploy",
    auth=auth,
    json=deploy_model_payload,
)

# Print the response
print(response.text)

{"task_id":"ElZQjo4B5vU2gwgcUbgf","task_type":"DEPLOY_MODEL","status":"COMPLETED"}


모델 테스트하기


In [6]:
test_model_payload = {"parameters": {"inputText": "바다에서 펼쳐지는 해적들의 꿈과 모험 이야기"}}

# Send the request
response = requests.post(
    base_url + "/models/" + "kGhNiI4BnEOV7ezVxzHF" + "/_predict",
    auth=auth,
    json=test_model_payload,
)

# Print the response
display(json.loads(response.text))

{'inference_results': [{'output': [{'name': 'sentence_embedding',
     'data_type': 'FLOAT32',
     'shape': [1536],
     'data': [0.0075683594,
      -0.80078125,
      0.0054016113,
      0.23828125,
      0.68359375,
      0.2109375,
      -0.12207031,
      -0.0005187988,
      -0.94140625,
      -0.84375,
      0.12011719,
      0.59375,
      -0.17382812,
      1.1953125,
      0.19335938,
      -0.40429688,
      -0.23242188,
      -0.14550781,
      -0.42382812,
      -0.38476562,
      0.41601562,
      -0.5234375,
      -0.15625,
      1.203125,
      -0.20996094,
      0.25976562,
      0.35546875,
      -1.21875,
      -0.029907227,
      0.18554688,
      -0.83203125,
      1.0703125,
      0.37109375,
      0.31054688,
      -0.04663086,
      -0.33398438,
      -0.13769531,
      -0.44335938,
      0.18066406,
      -0.056884766,
      -0.73828125,
      -0.74609375,
      -0.24707031,
      0.47070312,
      -0.056640625,
      0.48046875,
      0.75390625,
      0.0327

## Indexing


In [7]:
# aos_client.ingest.get_pipeline(id="embedding_pipeline")
# aos_client.indices.delete(index="hotels_index")
model_id = "ij1Qjo4B1FbzESchTZWr"

### Ingest Pipeline 생성


In [12]:
pipeline = {
    "description": "An neural search pipeline for hotel index - titan embedding g1 text",
    "processors": [
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "text": "vector_field",
                },
            }
        }
    ],
}

pipeline_id = "hotels_embedding_pipeline"
aos_client.ingest.put_pipeline(id=pipeline_id, body=pipeline)

{'acknowledged': True}

In [13]:
aos_client.ingest.get_pipeline(id=pipeline_id)

{'hotels_embedding_pipeline': {'description': 'An neural search pipeline for hotel index - titan embedding g1 text',
  'processors': [{'text_embedding': {'model_id': 'ij1Qjo4B1FbzESchTZWr',
     'field_map': {'text': 'vector_field'}}}]}}

### Index 생성


In [14]:
index_name = "hotels_index"

In [14]:
hotels_index = {
    "settings": {
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
        "index.knn": True,
        "default_pipeline": pipeline_id,
        "index.knn.space_type": "cosinesimil",
    },
    "mappings": {
        "properties": {
            "name": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "city": {
                "type": "keyword",
            },
            "rating": {"type": "integer"},
            "address": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "description": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "facilities": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "attractions": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "airport": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "text": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "map": {"type": "geo_point"},
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },
        }
    },
}

In [15]:
aos_client.indices.create(index=index_name, body=hotels_index)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'hotels_index'}

In [16]:
aos_client.indices.get(index=index_name)

{'hotels_index': {'aliases': {},
  'mappings': {'properties': {'address': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'airport': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'attractions': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'city': {'type': 'keyword'},
    'description': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'facilities': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'map': {'type': 'geo_point'},
    'name': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'rating': {'type': 'integer'},
    'text': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'vector_field': {'type': 'knn_vector',
     'store': True,
     'dimension': 1536,
     'method': {'engine': 

# Ingest Data


In [17]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = hotels_raw_df.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=10, thread_count=2, queue_size=2
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

In [18]:
# Refresh the index to make the changes visible
aos_client.indices.refresh(index=index_name)

{'_shards': {'total': 15, 'successful': 15, 'failed': 0}}

In [19]:
res = aos_client.search(index=index_name, body={"query": {"match_all": {}}})
print("Records found: %d " % res["hits"]["total"]["value"])
count = aos_client.count(index=index_name)
print(count)

Records found: 10000 
{'count': 11514, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


## 검색


### 키워드 검색


In [30]:
def keyword_search(query_text):
    query = {
        "size": 30,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["name", "address", "description", "facilities", "airport"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["name"],
            hit["_source"]["city"],
            hit["_source"]["address"],
            hit["_source"]["rating"],
            hit["_source"]["description"],
            hit["_source"]["attractions"],
            hit["_source"]["facilities"],
            hit["_source"]["airport"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result,
        columns=[
            "_score",
            "name",
            "city",
            "address",
            "raiting",
            "description",
            "attractions",
            "facilities",
            "airport",
        ],
    )
    display(query_result_df)

In [31]:
query_text = "윈드서핑"
# query_text = "클라이밍"
# query_text = "아침에 해가 뜨는 것을 볼 수 있는 숙소"
# query_text = "여름에 머물기 좋은 숙소"
# query_text = "겨울에 스키타러 가서 자기 좋은 숙소"
# query_text = "고기 구워먹을 수 있는 숙소"
# query_text = "한국 역사와 전통 문화를 배울 수 있는 장소"

In [32]:
keyword_search(query_text)

,_score,name,city,address,raiting,description,attractions,facilities,airport
0,5.521170,뜨거운 지붕 위의 고양이,양양,동산근길 66,3,"뜨거운 지붕 위의 고양이는 양양에 위치한 숙소로, 죽도해수욕장에서 300m, 인구해...","죽도해수욕장(300m), 인구해수욕장(1.5km), 장호해수욕장(1.9km), 대포...","발코니, 다트게임, 무료 WiFi, 테라스, 산 전망, 케이블 TV, 완비된 주방,...",양양국제공항
1,3.577612,뜨거운 지붕 위의 고양이,양양,동산근길 66,5,"양양에 위치한 뜨거운 지붕 위의 고양이는 죽도 해변에서 300m, 인구 해변에서 1...",정보가 업데이트 되지 않았습니다,"발코니, 다트 게임, 무료 WiFi, 정원, 공용 라운지",양양국제공항
2,3.577612,뜨거운 지붕 위의 고양이,양양,동산근길 66,3,"양양에 위치한 뜨거운 지붕 위의 고양이는 죽도 해변에서 300m, 인구 해변에서 1...",정보가 업데이트 되지 않았습니다,"발코니, 다트 게임, 무료 WiFi, 테라스, 산 전망, 거실, 케이블 TV, 주방...",양양국제공항


### Neural 검색 (Semantic)


In [33]:
def neural_search(query_text):
    query = {
        "size": 30,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
                "vector_field": {
                    "query_text": query_text,
                    "model_id": model_id,
                    "k": 30,
                }
            },
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["name"],
            hit["_source"]["city"],
            hit["_source"]["address"],
            hit["_source"]["rating"],
            hit["_source"]["description"],
            hit["_source"]["attractions"],
            hit["_source"]["facilities"],
            hit["_source"]["airport"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result,
        columns=[
            "_score",
            "name",
            "city",
            "address",
            "raiting",
            "description",
            "attractions",
            "facilities",
            "airport",
        ],
    )
    display(query_result_df)

In [34]:
neural_search(query_text)

,_score,name,city,address,raiting,description,attractions,facilities,airport
0,0.003419,윈드플라워,경주,경상북도 경주시 탐내길 18-6,3,경상북도 경주시에 위치한 윈드플라워는 경주 세계유산에서 10km 거리에 있으며 무료...,"석굴암(23km), 첨성대(2.4km)","에어컨, 완비된 주방, 평면 TV, 샤워 시설과 헤어드라이어가 있는 개인 욕실, 정...",포항국제공항(27km)
1,0.003399,오핑 펜션,양양,강원도 양양군 강현면 강현면 드윈나루2길 24-1,3,"설악해변 근처에 위치한 오핑 펜션은 설악해변에서 도보로 4분, 낙산사에서 차로 5분...","설악해변(0.3km), 낙산해변(2km), 의상대(2.4km), 낙산사(2.4km)...","주방, 냉장고, 스토브, 40인치 평면 TV, 무선 인터넷, 전자레인지, 전기 주전자",양양국제공항
2,0.003296,거제 골든 글로우 웨이브 펜션,거제,경상남도 거제시 일운면 망치3길 12,3,"거제(일운면)에 위치한 골든웨이브 펜션에 머무르면 구조라비치에서 3분, 거제조선해양...","구조라비치(3.1km), 와현해수욕장(4.4km), 거제조선해양문화관(7.7km),...","주방, 전자레인지, 냉장고/냉동고, LED TV, 무료 WiFi, 전기 주전자, 청...",김해국제공항
3,0.003237,가평 더 마운틴 슬로프 펜션,가평,경기도 가평군 북면 백둔로 510번길 130,3,가평 더 마운틴 슬로프 펜션에 머무시면 남송미술관과 명지산자연휴양림에 가깝습니다. ...,"남송미술관(1.7km), 명지산자연휴양림(4.5km), 연인산도립공원(7.2km),...",정보가 업데이트 되지 않았습니다,정보가 업데이트 되지 않았습니다
4,0.003202,오핑 펜션,양양,강원도 양양군 강현면 강현면 2길 24-1,3,"설악해변 인근에 위치한 오핑 펜션은 설악해변에서 도보로 4분, 낙산사에서 차로 5분...","설악해변(0.3km), 낙산해변(2km), 의상대(2.4km), 낙산사(2.4km)...","주방, 냉난방시설, 평면TV, 케이블TV, 무료WiFi, 전자레인지, 전기주전자",양양국제공항
5,0.003198,워킹 스카이 펜션,가평,경기도 가평군 가평읍 금대리 139-18,3,정보가 업데이트 되지 않았습니다,주요 관광지,제공된 정보가 없습니다,제공된 정보가 없습니다
6,0.003197,태안 웨스트윈드 펜션,태안,충청남도 태안군 안면읍 백사장1길 30-47,2,정보가 업데이트 되지 않았습니다,"삼봉해수욕장(2.4km/1.5마일), 간월암(8.2km/5.1마일), 몽산포해수욕장...",제공된 정보가 없습니다,청주국제공항(105km/65.3마일)
7,0.003191,휘슬 스톱 펜션,평창군,강원도 평창군 용평면 용평리 금당길 363,2,평창에 위치한 휘슬 스톱 펜션은 이효석 기념관과 복왕 불꽃공원에서 15분 거리에 있...,"이효석 기념관(6.9km), 복왕 불꽃공원(7.3km), 휘닉스 블루캐년(7.5km...","바/라운지, 에어컨, 평면 TV, 메모리폼 침대, 다운 이불, 이집트산 면 시트, ...",원주공항(39.9km)
8,0.003183,윈모텔,대구,대구 동구 신암남로 128,1,윈모텔은 대구에 위치한 객실에 에어컨이 마련되어 있습니다. 이 숙박 시설은 E-월드...,국채보상운동기념공원은 윈모텔에서 3.6km 거리에 있습니다.,"객실에는 전기 주전자가 비치되어 있으며, 모든 객실에는 냉장고가 마련되어 있습니다.",정보가 업데이트 되지 않았습니다
9,0.003174,램프 펜션,가평,경기도 가평군 조종면 와곡길 3-56,2,가평에 위치한 램프 펜션은 선힐 골프 클럽와 운악산 자연휴양림 근처에 있습니다. 이...,"선힐 골프 클럽(1.2km), 운악산 자연휴양림(4.3km), 전통주박물관 산사원(...",정보가 업데이트 되지 않았습니다,정보가 업데이트 되지 않았습니다


### 하이브리드 검색


In [37]:
def hybrid_search(query_text):
    query = {
        "size": 30,
        "_source": {"exclude": ["text", "vector_field"]},
        "query": {
            "hybrid": {
                "queries": [
                    {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["name", "address", "description", "facilities", "airport"],
                        }
                    },
                    {
                        "neural": {
                            "vector_field": {
                                "query_text": query_text,
                                "model_id": model_id,
                                "k": 30,
                            }
                        }
                    },
                ]
            }
        },
        "search_pipeline": {
            "description": "Post processor for hybrid search",
            "phase_results_processors": [
                {
                    "normalization-processor": {
                        "normalization": {"technique": "min_max"},
                        "combination": {
                            "technique": "arithmetic_mean",
                            "parameters": {"weights": [0.2, 0.8]},
                        },
                    }
                }
            ],
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["name"],
            hit["_source"]["city"],
            hit["_source"]["address"],
            hit["_source"]["rating"],
            hit["_source"]["description"],
            hit["_source"]["attractions"],
            hit["_source"]["facilities"],
            hit["_source"]["airport"],
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result,
        columns=[
            "_score",
            "name",
            "city",
            "address",
            "raiting",
            "description",
            "attractions",
            "facilities",
            "airport",
        ],
    )
    display(query_result_df)

In [38]:
hybrid_search(query_text)

,_score,name,city,address,raiting,description,attractions,facilities,airport
0,0.800000,윈드플라워,경주,경상북도 경주시 탐내길 18-6,3,경상북도 경주시에 위치한 윈드플라워는 경주 세계유산에서 10km 거리에 있으며 무료...,"석굴암(23km), 첨성대(2.4km)","에어컨, 완비된 주방, 평면 TV, 샤워 시설과 헤어드라이어가 있는 개인 욕실, 정...",포항국제공항(27km)
1,0.772616,오핑 펜션,양양,강원도 양양군 강현면 강현면 드윈나루2길 24-1,3,"설악해변 근처에 위치한 오핑 펜션은 설악해변에서 도보로 4분, 낙산사에서 차로 5분...","설악해변(0.3km), 낙산해변(2km), 의상대(2.4km), 낙산사(2.4km)...","주방, 냉장고, 스토브, 40인치 평면 TV, 무선 인터넷, 전자레인지, 전기 주전자",양양국제공항
2,0.635596,거제 골든 글로우 웨이브 펜션,거제,경상남도 거제시 일운면 망치3길 12,3,"거제(일운면)에 위치한 골든웨이브 펜션에 머무르면 구조라비치에서 3분, 거제조선해양...","구조라비치(3.1km), 와현해수욕장(4.4km), 거제조선해양문화관(7.7km),...","주방, 전자레인지, 냉장고/냉동고, LED TV, 무료 WiFi, 전기 주전자, 청...",김해국제공항
3,0.556314,가평 더 마운틴 슬로프 펜션,가평,경기도 가평군 북면 백둔로 510번길 130,3,가평 더 마운틴 슬로프 펜션에 머무시면 남송미술관과 명지산자연휴양림에 가깝습니다. ...,"남송미술관(1.7km), 명지산자연휴양림(4.5km), 연인산도립공원(7.2km),...",정보가 업데이트 되지 않았습니다,정보가 업데이트 되지 않았습니다
4,0.510740,오핑 펜션,양양,강원도 양양군 강현면 강현면 2길 24-1,3,"설악해변 인근에 위치한 오핑 펜션은 설악해변에서 도보로 4분, 낙산사에서 차로 5분...","설악해변(0.3km), 낙산해변(2km), 의상대(2.4km), 낙산사(2.4km)...","주방, 냉난방시설, 평면TV, 케이블TV, 무료WiFi, 전자레인지, 전기주전자",양양국제공항
5,0.505623,워킹 스카이 펜션,가평,경기도 가평군 가평읍 금대리 139-18,3,정보가 업데이트 되지 않았습니다,주요 관광지,제공된 정보가 없습니다,제공된 정보가 없습니다
6,0.503953,태안 웨스트윈드 펜션,태안,충청남도 태안군 안면읍 백사장1길 30-47,2,정보가 업데이트 되지 않았습니다,"삼봉해수욕장(2.4km/1.5마일), 간월암(8.2km/5.1마일), 몽산포해수욕장...",제공된 정보가 없습니다,청주국제공항(105km/65.3마일)
7,0.495545,휘슬 스톱 펜션,평창군,강원도 평창군 용평면 용평리 금당길 363,2,평창에 위치한 휘슬 스톱 펜션은 이효석 기념관과 복왕 불꽃공원에서 15분 거리에 있...,"이효석 기념관(6.9km), 복왕 불꽃공원(7.3km), 휘닉스 블루캐년(7.5km...","바/라운지, 에어컨, 평면 TV, 메모리폼 침대, 다운 이불, 이집트산 면 시트, ...",원주공항(39.9km)
8,0.485362,윈모텔,대구,대구 동구 신암남로 128,1,윈모텔은 대구에 위치한 객실에 에어컨이 마련되어 있습니다. 이 숙박 시설은 E-월드...,국채보상운동기념공원은 윈모텔에서 3.6km 거리에 있습니다.,"객실에는 전기 주전자가 비치되어 있으며, 모든 객실에는 냉장고가 마련되어 있습니다.",정보가 업데이트 되지 않았습니다
9,0.473053,램프 펜션,가평,경기도 가평군 조종면 와곡길 3-56,2,가평에 위치한 램프 펜션은 선힐 골프 클럽와 운악산 자연휴양림 근처에 있습니다. 이...,"선힐 골프 클럽(1.2km), 운악산 자연휴양림(4.3km), 전통주박물관 산사원(...",정보가 업데이트 되지 않았습니다,정보가 업데이트 되지 않았습니다


In [ ]:
mapping = {
    "settings": {
        "index": {
            "knn": True,
            "knn.space_type": "consinesimil",
        }
    }
    "ma"
}